# Setup

### Imports

In [1]:
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.functions import col, explode, to_timestamp

### Creating a Spark Session

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
  .appName('Movielens Data Preparation')\
  .getOrCreate()

### Defining Data Folder

In [3]:
# Path where the raw data is stored
data = "gs://ca4022-recsys-data/data/movielens_small/"

# Loading the Data + Cleaning

The dataset which I have chosen to use for the purpose of building a recommender system is a smaller version of the [**MovieLens** dataset](https://grouplens.org/datasets/movielens/). Originally, I experimented with the full dataset but the scale of the data made cross-validation of ALS models infeasible. The dataset describes 5-star rating and free-text tagging activity from MovieLens, a movie recommendation service. It contains just over 100,000 ratings and over 3,500 tags across over 9,742 movies. The dataset was created by 610 users between March 1996 and September 2018. Users were selected at random for inclusion and all selected users have rated at least 20 movies. Below, we load in the data and process it accordingly.

### Movies Data

In [4]:
df_movies = spark.read.csv(data + "movies.csv", header=True)

In [5]:
df_movies.limit(10).toPandas()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [6]:
df_movies.printSchema()

root
 |-- movieId: string (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



The *movieId* column should be an integer so we cast it as so.

In [7]:
df_movies = df_movies.withColumn('movieId', col('movieId').cast('integer'))

### Ratings Data

In [9]:
df_ratings = spark.read.csv(data + "ratings.csv", header=True)

In [10]:
df_ratings.limit(10).toPandas()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
5,1,70,3.0,964982400
6,1,101,5.0,964980868
7,1,110,4.0,964982176
8,1,151,5.0,964984041
9,1,157,5.0,964984100


In [11]:
df_ratings.printSchema()

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- timestamp: string (nullable = true)



Again, the *userId* and *movieId* columns should be integers and the *rating* column should be a float, so I'll cast them as so. The timestamp information associated with each rating will not be used in the recommender system so this column can be dropped also.

In [12]:
df_ratings = df_ratings.\
                withColumn('userId', col('userId').cast('integer')).\
                withColumn('movieId', col('movieId').cast('integer')).\
                withColumn('rating', col('rating').cast('float')).\
                drop('timestamp')

In [13]:
df_ratings.limit(10).toPandas()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
5,1,70,3.0
6,1,101,5.0
7,1,110,4.0
8,1,151,5.0
9,1,157,5.0


For ease, we can simply join the ratings and movies tables together.

In [14]:
df_mov_ratings = df_ratings.join(df_movies, ['movieId'], 'left')
df_mov_ratings.limit(10).toPandas()

,movieId,userId,rating,title,genres
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,3,1,4.0,Grumpier Old Men (1995),Comedy|Romance
2,6,1,4.0,Heat (1995),Action|Crime|Thriller
3,47,1,5.0,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,50,1,5.0,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
5,70,1,3.0,From Dusk Till Dawn (1996),Action|Comedy|Horror|Thriller
6,101,1,5.0,Bottle Rocket (1996),Adventure|Comedy|Crime|Romance
7,110,1,4.0,Braveheart (1995),Action|Drama|War
8,151,1,5.0,Rob Roy (1995),Action|Drama|Romance|War
9,157,1,5.0,Canadian Bacon (1995),Comedy|War


### Storing the Cleaned Data

In [15]:
df_mov_ratings.write.option("header", "true").option("sep", ",").csv(data + "clean/movie_ratings_small.csv")